1) Write Python code and use MapReduct to count occurrences of each word in the first text file (file.txt). How many times each word is repeated?

In [13]:
import sys
from collections import Counter


def mapper(line):
    
    words = line.strip().split()
    for word in words:
        yield (word.lower(), 1)


In [15]:
def reducer(word, counts):
    
    count = sum(counts)
    yield (word, count)

In [ ]:
with open('file1.txt', 'r') as f:
  
    word_counts = Counter()
    
    for line in f:
        
        mapped = map(mapper, [line])
       
        grouped = {}
        for key, value in mapped:
            grouped.setdefault(key, []).append(value)
        
        reduced = [reducer(key, value) for key, value in grouped.items()]
        
        for word, count in reduced:
            word_counts[word] += count

In [ ]:
# Printring word counts
for word, count in word_counts.items():
    print(word, count)


2) From the second text file (file2.txt), write Python code and use MapReduct to count how many times non-English words (names, places, spells etc.) were used. List those words and how many times each was repeated.
There are multiple ways of doing this.  You can use pyenchant (https://pypi.org/project/pyenchant/), pyspellchecker (https://pyspellchecker.readthedocs.io/en/latest/) or just download a list of words (http://www.gwicks.net/dictionaries.htm) and search through them.

In [ ]:
#using pyspellchecker

import pyspark
from pyspark import SparkContext

from spellchecker import SpellChecker


sc = SparkContext("local", "Non-English Words Counter")


lines = sc.textFile("file2.txt")



In [ ]:
def extract_non_english_words(line):
    
    spell = SpellChecker(language='en')
    
   
    words = line.split()
    
    non_english_words = filter(lambda word: not spell.check(word), words)
   
    return [(word.lower(), 1) for word in non_english_words]


In [ ]:

#reduce function
def count_words(a, b):
    return a + b

In [ ]:
non_english_words_counts = lines.flatMap(extract_non_english_words) \
                                .reduceByKey(count_words) \
                                .collect()

In [ ]:
# printint the results
for word, count in non_english_words_counts:
    print(f"{word}: {count}"